
<span type="title">复合模式和MVC</span> | <span type="update">2018-09-10</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍组合使用多种设计模式的复合模式以及在 Java GUI 和 Java EE 经常被使用的 MVC 模型-视图-控制器模式。</p></span>

# 复合模式

复合模式就是混合使用多种设计模式。根据需求，我们将设计模式分为两大类，统一接口的设计模式和提供服务的设计模式。

对于前者，有装饰器（添加责任）、适配器（适配标准）、抽像工厂（提供标准）、命令模式（唯一标准）、外观模式（简化标准）、组合模式（统一标准）、迭代器模式（用于获取容器的简化标准）。

对于后者，可细分为基于组合+委托的与基于继承的。基于组合的有：策略模式（强调灵活组配）、状态模式（强调状态切换）、观察者模式（强调状态统一），基于继承的有工厂方法（创建对象权交给子类）、模板方法（算法具体实现交给子类）。

此外，还有单件模式、简单工厂等特殊的设计模式。

下面是一个输出 “Hello, World!” 的代码，使用了八种设计模式，长达267行。虽然我们都知道这是过度设计，但是，其实每种设计模式的使用都有理由，很显然我们在这里没有把握好这个度的问题。这是一个绝妙的例子，用来展示复合模式。

```java
//组合模式
Word word = new Word();
word.add(new UpperLetter(new BasicLetter('h')));
word.add(new BasicLetter('e'));
word.add(new BasicLetter('l'));   
word.add(new BasicLetter('l'));
word.add(new BasicLetter('o'));
//迭代器模式（内部实现）
word.printLetter(writer);
//装饰器模式（三层装饰器，分别用来检查全角字符以及转换大些）
new UpperLetter(new CheckedLetter(new BasicLetter('，'))).printLetter(writer);
//策略模式（输出World这个单词）
Strategy worldStrategy = new Strategy() {
    @Override
    void initLetters() {
//观察者模式（用于给大些字母前自动添加空格，观察者打印了一个空格）
        LetterObserver spacer = new Spacer(writer);
        UpperLetter letter = new UpperLetter(new CheckedLetter(new BasicLetter('w')));
        letter.addObserver(spacer);
        this.letters.add(letter);
        this.letters.add(new CheckedLetter(new BasicLetter('o')));
        this.letters.add(new CheckedLetter(new BasicLetter('r')));
        this.letters.add(new CheckedLetter(new BasicLetter('l')));
        this.letters.add(new CheckedLetter(new BasicLetter('d')));
    }
};
//模板方法模式(StrategyFactory继承了CheckFactory的模板方法，后者实现了setList，并对工厂对象进行了检查)
Word5Factory factory = new Strategy5Factory(worldStrategy);
//工厂模式（抽象工厂模式和生成Letter对象的工厂方法模式）
factory.get1Letter().printLetter(writer);
factory.get2Letter().printLetter(writer);
factory.get3Letter().printLetter(writer);
factory.get4Letter().printLetter(writer);
factory.get5Letter().printLetter(writer);
//适配器模式（用来将符号类型适配成为Letter类型）
Letter symbol = new SymbolAdaptor(new Symbol('!'));
symbol.printLetter(writer);
```

## 面向对象的接口实现

首先，我将字符串拆分成字符，然后定义一个类型来存放这些字符串。

```java
//基本类型接口
interface Letter{
    Character getCharacter();
    void printLetter(PrintWriter writer);
}
//基本类型的实现
class BasicLetter implements Letter {
    private Character character;
    public BasicLetter(Character character) {
        this.character = character;
    }
    @Override
    public Character getCharacter() {
        return character;
    }
    @Override
    public void printLetter(PrintWriter writer) { writer.print(character); }
}
```

## 装饰器模式的使用

很显然，我们可以对这个类型添加一些新的模式，比如检查大小写，检查是否全角，使用装饰器来添加责任，创建一个实例来保存原有对象，并且实现其接口，操纵其行为来完成服务。

```java
//检查中文字符的基本类型的装饰器
class CheckedLetter implements Letter{
    private Letter letter;
    public CheckedLetter(Letter letter) {
        this.letter = letter;
    }
    @Override
    public Character getCharacter() {
        switch (letter.getCharacter()) {
            case '。':
                return new BasicLetter('.').getCharacter();
            case '，':
                return new BasicLetter(',').getCharacter();
            default:
                return letter.getCharacter();
        }
    }
    @Override
    public void printLetter(PrintWriter writer) {
        switch (letter.getCharacter()) {
            case '。':
                writer.print('.'); return;
            case '，':
                writer.print(','); return;
        }
        letter.printLetter(writer);
    }
}
//强制大写的基本类型的装饰器
class UpperLetter implements Letter, LetterObservable  {
    LetterObservable observable;
    @Override
    public void addObserver(LetterObserver observer) {
        observable.addObserver(observer);
    }
    @Override
    public void notifyObserver() {
        observable.notifyObserver();
    }
    private Letter letter;
    public UpperLetter(Letter letter) {
        this.letter = letter;
        this.letter = new BasicLetter(Character.toUpperCase(this.letter.getCharacter()));
        observable = new AbstractLetterObservable(this);
    }
    @Override
    public void printLetter(PrintWriter writer) {
        notifyObserver();
        letter.printLetter(writer);
    }
    @Override
    public Character getCharacter() {
        return letter.getCharacter();
    }
}
```

## 适配器类型的使用

我们现在遇到了一些问题，有一种不兼容的类型，使用适配器可以让它兼容我们的 Letter 类。

```java
//外来类型
class Symbol {
    private Character character;
    public Symbol(Character character) { this.character = character; }
    public void printLetter() { System.out.println(character); }
    public Character getCharacter() { return this.character; }
}
//外来类型的基本类型适配器
class SymbolAdaptor implements Letter {
    private Symbol symbol;
    public SymbolAdaptor(Symbol symbol) { this.symbol = symbol; }
    @Override
    public Character getCharacter() {
        return symbol.getCharacter();
    }
    @Override
    public void printLetter(PrintWriter writer) {
        writer.print(symbol.getCharacter());
    }
}
```

## 抽像工厂的使用

很显然，如果我们new几个对象，然后调用 printLetter，这样很容易搞错，因此使用抽像工厂来管理创建对象实例，这样我们的客户端代码就能复用了。

```java
//一次组装多个基本类型的工厂接口
interface Word5Factory {
    Letter get1Letter();
    Letter get2Letter();
    Letter get3Letter();
    Letter get4Letter();
    Letter get5Letter();
}
```

## 模板方法的使用

对于单个字母，我们使用装饰器来进行责任添加，那么对于工厂呢？我们很容易就想到，使用交给子类的方法来让子类完成返回对象的责任，这似乎也能看作一种工厂方法模式。不过，setList返回的是一组对象。

```java
//一次组装多个基本类型的工厂的抽象接口
abstract class AbstractWord5Factory implements Word5Factory {
    //这里必须使用abstract，否则super会执行此处的setList，那样就错误了，我们想要的是子类提供的实现
    protected ArrayList<Letter> list = new ArrayList();
    AbstractWord5Factory(Letter...args) {
        setList(args);
    }
    abstract void setList(Letter...args);
    @Override
    public Letter get1Letter() {
        return list.get(0);
    }
    @Override
    public Letter get2Letter() {
        return list.get(1);
    }
    @Override
    public Letter get3Letter() {
        return list.get(2);
    }
    @Override
    public Letter get4Letter() {
        return list.get(3);
    }
    @Override
    public Letter get5Letter() {
        return list.get(4);
    }
}
//对于AbstractWord5Factory进行逐字检查的交给子类的模板方法模式
class CheckedWord5Factory extends AbstractWord5Factory implements Word5Factory {
   public CheckedWord5Factory(Letter...args) { super(args); }
    @Override
    void setList(Letter... args) {
       for (int i=0;i < args.length;i++) { list.add(new CheckedLetter(args[i])); }
    }
}
```

我们创建了一个CheckedWord的模板方法子类，这可以看作是“工厂的装饰器”，我们遍历其中所有的生产对象，然后使用装饰器添加了责任。

## 策略模式的使用

现在的问题是，工厂需要传递参数进去，我们需要传递单词，这些单词可以使用策略模式进行传递。

```java
//策略的抽象类
abstract class Strategy {
    public Strategy() { initLetters(); }
    ArrayList<Letter> letters = new ArrayList<>();
    abstract void initLetters();
    public Letter[] getLetters() {
        Letter[] res = new Letter[letters.size()];
        for (int i = 0; i < letters.size(); i++) {
            res[i] = letters.get(i);
        }
        return res;
    }
}
//使用了策略模式的工厂模式
class Strategy5Factory extends CheckedWord5Factory {
    public Strategy5Factory(Strategy strategy) {
        setList(strategy.getLetters());
    }
    @Override
    void setList(Letter... args) {
        for (int i=0;i < args.length;i++) { list.add(args[i]); }
    }
}
```

其实这里的策略模式也可以直接使用模板方法来交给子类一起完成，换句话说，使用继承和使用组合在这里可以互相替代。我们将策略模式和工厂模式结合起来，用于指示工厂生产的产品。这个使用组合的工厂类完全可以替代抽像的使用“使用委托+继承”的抽像工厂类 AbstractWord5Factory。

## 组合模式的使用

## 迭代器模式的使用

我们除了使用工厂，还可以使用组合，组合使用透明性换取了一致性，组合本身可以当作其子对象使用。这样的话，我们可以搞出来一个组合了多个Letter类的Letter类。这样调用 printLetter 方法就会逐个遍历进行打印，同时没有新增任何接口。

注意在这里我们使用了迭代器模式。

```java
//基本类型组合形成的复合类型，内置其保存基本类型对象的迭代器
class Word implements Letter {
    ArrayList<Letter> letters = new ArrayList<>();
    public void add(Letter letter) {
        letters.add(letter);
    }
    //不合适的方法，不去实现
    @Override
    public Character getCharacter() {
        throw new RuntimeException("UNSEEMLINESS CALL");
    }
    @Override
    public void printLetter(PrintWriter writer) {
        Iterator<Letter> letterIterator = letters.iterator();
        while (letterIterator.hasNext()) {
            letterIterator.next().printLetter(writer);
        }
    }
}
```

## 观察者模式的使用

最后，我们还有一个需求，当字母是大写的时候，我们需要在其前面添加空格或者做别的事情，这个时候使用观察者模式很方便，对于UpperLetter类使用观察者模式，让其充当观察对象，然后新建观察者，比如我创建了一个用来添加空格的观察者，将观察者注册到观察对象，并且在需要打印的时候调用更新以打印；

```java
//观察者模式的观察者和被观察者的接口
interface LetterObserver {
    void update(Letter letter);
}
interface LetterObservable {
    void addObserver(LetterObserver observer);
    void notifyObserver();
}
//观察者模式的观察者、被观察者
class Spacer implements LetterObserver {
    PrintWriter writer;
    Spacer(PrintWriter writer) { this.writer = writer; }
    public void update(Letter letter) {
        writer.print(" ");
    }
}
```

注意，在这里我新建了一个 AbstractLetterObservable对象，然后将其放入被观察对象中，产生行为，这种方式很像策略模式，不过为了简化代码，我们直接在构造器中添加了这个对象，让其被保存成UpperLetter类的实例。

```java
class AbstractLetterObservable implements LetterObservable {
    ArrayList<LetterObserver> observers = new ArrayList<>();
    Letter letter;
    public AbstractLetterObservable(Letter letter) {
        this.letter = letter;
    }
    public void addObserver(LetterObserver observer) {
        observers.add(observer);
    }
    public void notifyObserver() {
        for (LetterObserver observer : observers) {
            observer.update(letter); }
    }
}
```

这里再看一下UpperLetter类的实现，注意 printLetter 前我们通知了观察者，观察者在 update 方法中添加了空格。

```java
class UpperLetter implements Letter, LetterObservable  {
    LetterObservable observable;
    @Override
    public void addObserver(LetterObserver observer) {
        observable.addObserver(observer);
    }
    @Override
    public void notifyObserver() {
        observable.notifyObserver();
    }
    private Letter letter;
    public UpperLetter(Letter letter) {
        this.letter = letter;
        this.letter = new BasicLetter(Character.toUpperCase(this.letter.getCharacter()));
        observable = new AbstractLetterObservable(this);
    }
    @Override
    public void printLetter(PrintWriter writer) {
        notifyObserver();
        letter.printLetter(writer);
    }
    @Override
    public Character getCharacter() {
        return letter.getCharacter();
    }
}
```

```java
import java.io.PrintWriter;
import java.util.*;

public class StringDemo {
    public static void main(String[] args) {
        try (PrintWriter writer = new PrintWriter(System.out)) {
            Long start = System.nanoTime();
            writer.print("Hello, World!");
            Long timeNow = System.nanoTime()-start;
            writer.printf(" :: Time usage: %d s, %d ms, %d us, %d ns\n",
                    timeNow/1000_000_000,timeNow/1000_000,timeNow/1000, timeNow);
            start = System.nanoTime();
            //组合模式
            Word word = new Word();
            word.add(new UpperLetter(new BasicLetter('h')));
            word.add(new BasicLetter('e'));
            word.add(new BasicLetter('l'));   
            word.add(new BasicLetter('l'));
            word.add(new BasicLetter('o'));
            //迭代器模式（内部实现）
            word.printLetter(writer);
            //装饰器模式（三层装饰器，分别用来检查全角字符以及转换大些）
            new UpperLetter(new CheckedLetter(new BasicLetter('，'))).printLetter(writer);
            //策略模式（输出World这个单词）
            Strategy worldStrategy = new Strategy() {
                @Override
                void initLetters() {
            //观察者模式（用于给大些字母前自动添加空格，观察者打印了一个空格）
                    LetterObserver spacer = new Spacer(writer);
                    UpperLetter letter = new UpperLetter(new CheckedLetter(new BasicLetter('w')));
                    letter.addObserver(spacer);
                    this.letters.add(letter);
                    this.letters.add(new CheckedLetter(new BasicLetter('o')));
                    this.letters.add(new CheckedLetter(new BasicLetter('r')));
                    this.letters.add(new CheckedLetter(new BasicLetter('l')));
                    this.letters.add(new CheckedLetter(new BasicLetter('d')));
                }
            };
            //模板方法模式(StrategyFactory继承了CheckFactory的模板方法，后者实现了setList，并对工厂对象进行了检查)
            Word5Factory factory = new Strategy5Factory(worldStrategy);
            //工厂模式（抽象工厂模式和生成Letter对象的工厂方法模式）
            factory.get1Letter().printLetter(writer);
            factory.get2Letter().printLetter(writer);
            factory.get3Letter().printLetter(writer);
            factory.get4Letter().printLetter(writer);
            factory.get5Letter().printLetter(writer);
            //适配器模式（用来将符号类型适配成为Letter类型）
            Letter symbol = new SymbolAdaptor(new Symbol('!'));
            symbol.printLetter(writer);

            timeNow = System.nanoTime()-start;
            writer.printf(" :: Time usage: %d s, %d ms, %d us, %d ns\n",
                    timeNow/1000_000_000,timeNow/1000_000,timeNow/1000,  timeNow);
        }
    }
}
```

## 混合使用设计模式

```java
import java.io.PrintWriter;
import java.util.*;

public class StringDemo {
    public static void main(String[] args) {
        try (PrintWriter writer = new PrintWriter(System.out)) {
            Long start = System.nanoTime();
            writer.print("Hello, World!");
            Long timeNow = System.nanoTime()-start;
            writer.printf(" :: Time usage: %d s, %d ms, %d us, %d ns\n",
                    timeNow/1000_000_000,timeNow/1000_000,timeNow/1000, timeNow);
            start = System.nanoTime();
            //组合模式
            Word word = new Word();
            word.add(new UpperLetter(new BasicLetter('h')));
            word.add(new BasicLetter('e'));
            word.add(new BasicLetter('l'));   
            word.add(new BasicLetter('l'));
            word.add(new BasicLetter('o'));
            //迭代器模式（内部实现）
            word.printLetter(writer);
            //装饰器模式（三层装饰器，分别用来检查全角字符以及转换大些）
            new UpperLetter(new CheckedLetter(new BasicLetter('，'))).printLetter(writer);
            //策略模式（输出World这个单词）
            Strategy worldStrategy = new Strategy() {
                @Override
                void initLetters() {
            //观察者模式（用于给大些字母前自动添加空格，观察者打印了一个空格）
                    LetterObserver spacer = new Spacer(writer);
                    UpperLetter letter = new UpperLetter(new CheckedLetter(new BasicLetter('w')));
                    letter.addObserver(spacer);
                    this.letters.add(letter);
                    this.letters.add(new CheckedLetter(new BasicLetter('o')));
                    this.letters.add(new CheckedLetter(new BasicLetter('r')));
                    this.letters.add(new CheckedLetter(new BasicLetter('l')));
                    this.letters.add(new CheckedLetter(new BasicLetter('d')));
                }
            };
            //模板方法模式(StrategyFactory继承了CheckFactory的模板方法，后者实现了setList，并对工厂对象进行了检查)
            Word5Factory factory = new Strategy5Factory(worldStrategy);
            //工厂模式（抽象工厂模式和生成Letter对象的工厂方法模式）
            factory.get1Letter().printLetter(writer);
            factory.get2Letter().printLetter(writer);
            factory.get3Letter().printLetter(writer);
            factory.get4Letter().printLetter(writer);
            factory.get5Letter().printLetter(writer);
            //适配器模式（用来将符号类型适配成为Letter类型）
            Letter symbol = new SymbolAdaptor(new Symbol('!'));
            symbol.printLetter(writer);

            timeNow = System.nanoTime()-start;
            writer.printf(" :: Time usage: %d s, %d ms, %d us, %d ns\n",
                    timeNow/1000_000_000,timeNow/1000_000,timeNow/1000,  timeNow);
        }
    }
}
```